## 1. 라이브러리 및 기업리스트 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup as bs
import xmltodict
import json
import dart_fss as dart

In [ ]:
kosdaq_list = pd.read_excel('코스닥기업리스트.xlsx', index_col=1)

## 2. DART API 사용을 위해 기업 고유번호 매칭

In [53]:
key = "87936e8a4f731de42dbc51060807f54a281a8f98" 
url = f"http://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}"
res_url = requests.get(url)

ZipFile(BytesIO(res_url.content)).extractall("./data")
xml = open("./data/CORPCODE.xml", "r", encoding='utf-8').read()
dict_type = xmltodict.parse(xml)

corp_dict = {}
for company in dict_type['result']['list']:
    if company['stock_code']:
        corp_dict[int(company['stock_code'])] = company['corp_code']
        
kosdaq_list['고유번호'] = None
for idx in kosdaq_list.index:
    kosdaq_list.loc[idx, '고유번호'] = corp_dict[idx]

In [55]:
# 기업별 고유번호 포함한 데이터프레임 저장
kosdaq_list.reset_index(inplace=True)
kosdaq_list.set_index('고유번호', drop=True, inplace=True)
# kosdaq_list.to_excel('kosdaq_list.xlsx')
# kosdaq_list

## 3. 기업 개황정보 크롤링

In [226]:
kosdaq_list['사업자등록번호'], kosdaq_list['주소'], kosdaq_list['업종코드'], kosdaq_list['설립일'] = None, None, None, None

for idx in kosdaq_list.index:
    res_url = requests.get(f'https://opendart.fss.or.kr/api/company.json?crtfc_key={key}&corp_code={idx}')
    company_info = json.loads(res_url.text)
    kosdaq_list.loc[idx, '사업자등록번호'] = company_info['bizr_no']
    kosdaq_list.loc[idx, '주소'] = company_info['adres']
    kosdaq_list.loc[idx, '업종코드'] = company_info['induty_code']
    kosdaq_list.loc[idx, '설립일'] = company_info['est_dt']

In [61]:
kosdaq_list.index = list(map(str, kosdaq_list.index))
# kosdaq_list.to_excel('kosdaq_info.xlsx')
# kosdaq_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,주소,업종코드,설립일
01508855,369370,블리츠웨이,"인형,장난감 및 오락용품 제조업",기업 인수 및 합병,2020-12-23,12월,배성웅,NaN,서울특별시,서울특별시 영등포구 국회대로 638 블리츠웨이빌딩,334,20201020
00921916,357550,석경에이티,기초 화학물질 제조업,바이오 및 전기전자 나노 소재,2020-12-23,12월,임형섭,http://www.sukgyung.com,경기도,경기도 안산시 단원구 별망로459번길 24,20129,20001218
01276026,314130,지놈앤컴퍼니,기초 의약물질 및 생물학적 제제 제조업,"마이크로바이옴 기반 치료제, 화장품, 건강기능식품",2020-12-23,12월,"배지수, 박한수, 서영진 (각자 대표이사)",http://genomecom.co.kr,경기도,"경기도 성남시 분당구 판교로255번길 35, B동 8층(삼평동, 판교실리콘파크)",211,20150924
01137295,335810,프리시젼바이오,의료용 기기 제조업,체외진단 기기 및 시약,2020-12-22,12월,김한신,http://precision-bio.com/,대전광역시,대전시 유성구 테크노2로 306,27199,20090928
01405451,347860,알체라,소프트웨어 개발 및 공급업,"얼굴인식 AI, 이상상황 감지 AI 등",2020-12-21,12월,황영규,http://www.alchera.ai,경기도,경기도 성남시 분당구 판교로256번길 25 7층(삼평동),58222,20160603
...,...,...,...,...,...,...,...,...,...,...,...,...
00165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,1989-12-15,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,부산광역시 강서구 녹산산단27로 97,241,19780318
00156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,1989-09-06,12월,공동대표이사 이준효 공동대표이사 소우에이이치로,http://www.sbik.co.kr,서울특별시,서울특별시 강남구 테헤란로 509엔씨타워1 14층,64209,19861124
00110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,1989-03-14,12월,김형준,http://www.geminivc.co.kr,서울특별시,서울특별시 강남구 테헤란로 447 KB우준타워 KB우준타워 17층,64209,19861127
00136642,19590,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,1989-03-14,12월,홍성혁,http://www.m-vc.co.kr,서울특별시,서울특별시 강남구 테헤란로87길 36 도심공항타워 2503호,64209,19861202


## 4. 기업 사업보고서 주주현황 및 감사의견 크롤링

In [234]:
year = 2021
#kosdaq_list['최대주주_성명'], kosdaq_list['최대주주_관계'], kosdaq_list['최대주주_주식종류'], kosdaq_list['최대주주_기초주식수'], kosdaq_list['최대주주_기초주식지분율'], kosdaq_list['최대주주_기말주식수'], kosdaq_list['최대주주_기말주식지분율'] = None, None, None, None, None, None, None
#kosdaq_list['소액주주수'], kosdaq_list['주주수'], kosdaq_list['소액주주비율'], kosdaq_list['소액주주_주식수'], kosdaq_list['총주식수'], kosdaq_list['소액주주_주식지분율'] = None, None, None, None, None, None
kosdaq_list['감사의견'], kosdaq_list['감사의견_강조사항'], kosdaq_list['감사의견_핵심감사사항'] = None, None, None

for idx in kosdaq_list.index:
    #res_url_1 = requests.get(f'https://opendart.fss.or.kr/api/hyslrSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011')  
    #res_url_2 = requests.get(f'https://opendart.fss.or.kr/api/mrhlSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011')  
    res_url_3 = requests.get(f'https://opendart.fss.or.kr/api/accnutAdtorNmNdAdtOpinion.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011')  
    
    json_reports = json.loads(res_url_3.text)
    if json_reports.get('list'):
#         adt_opinion, emphs_matter, core_adt_matter = '', '', ''
#         for company_report in json_reports['list']:
#             if company_report.get('relate'):
#             adt_opinion += company_report['adt_opinion'] + ' '
#             emphs_matter += company_report['emphs_matter'] + ' '
#             core_adt_matter += company_report['core_adt_matter'] + ' '

#         kosdaq_list.loc[idx, '최대주주_성명'] = nm
#         kosdaq_list.loc[idx, '최대주주_관계'] = relate
#         kosdaq_list.loc[idx, '최대주주_주식종류'] = stock_knd
#         kosdaq_list.loc[idx, '최대주주_기초주식수'] = bsis_posesn_stock_co
#         kosdaq_list.loc[idx, '최대주주_기초주식지분율'] = bsis_posesn_stock_qota_rt
#         kosdaq_list.loc[idx, '최대주주_기말주식수'] = trmend_posesn_stock_co
#         kosdaq_list.loc[idx, '최대주주_기말주식지분율'] = trmend_posesn_stock_qota_rt

#             kosdaq_list.loc[idx, '소액주주수'] = company_report['shrholdr_co']
#             kosdaq_list.loc[idx, '주주수'] = company_report['shrholdr_tot_co'] 
#             kosdaq_list.loc[idx, '소액주주비율'] = company_report['shrholdr_rate']
#             kosdaq_list.loc[idx, '소액주주_주식수'] = company_report['hold_stock_co']
#             kosdaq_list.loc[idx, '총주식수'] = company_report['stock_tot_co']
#             kosdaq_list.loc[idx, '소액주주_주식지분율'] = company_report['hold_stock_rate']

        kosdaq_list.loc[idx, '감사의견'] = json_reports['list'][0]['adt_opinion']
        kosdaq_list.loc[idx, '감사의견_강조사항'] = json_reports['list'][0]['emphs_matter']
        kosdaq_list.loc[idx, '감사의견_핵심감사사항'] = json_reports['list'][0]['core_adt_matter']

In [236]:
kosdaq_list.to_excel('kosdaq_report.xlsx')
kosdaq_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1388 entries, 01508855 to 00163318
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   종목코드          1388 non-null   int64         
 1   회사명           1388 non-null   object        
 2   업종            1388 non-null   object        
 3   주요제품          1385 non-null   object        
 4   상장일           1388 non-null   datetime64[ns]
 5   결산월           1388 non-null   object        
 6   대표자명          1388 non-null   object        
 7   홈페이지          1324 non-null   object        
 8   지역            1388 non-null   object        
 9   주소            1388 non-null   object        
 10  업종코드          1388 non-null   object        
 11  설립일           1388 non-null   object        
 12  최대주주_성명       1387 non-null   object        
 13  최대주주_관계       1387 non-null   object        
 14  최대주주_주식종류     1387 non-null   object        
 15  최대주주_기초주식수    1387 non-null   ob